In [31]:
#only need once
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

In [32]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import functools
import os
import re
from gensim.summarization import keywords
import numpy as np
import pandas as pd
import glob
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import codecs
import pickle

#global variable
STOPWORDS = set(stopwords.words("english"))
STEMMER = SnowballStemmer("english")

In [99]:
def read_txt(txtfile):
    with open(txtfile, 'r') as myfile:
        text=myfile.read()
    myfile.close()
    return text

#pre-con: txt is a raw text with \n
#post-con: returns text with global redundant words removed
def remove_global_redundancies(txt):
    txt = re.sub("Figure [0-9]+\..*?Source:.*?\n", "", txt, re.MULTILINE, re.DOTALL)
    txt = re.sub("Figure [0-9]+\..*|figure [0-9]+\..*|Source:.*|source:.*", "", txt)
    txt = re.sub("[0-9]+, Citi Research", "", txt)
    txt = re.sub(".*[0-9]+.*?\nCitigroup.*", "", txt)
    txt = re.sub(".*Citi GPS.*", "", txt)
    txt = re.sub("Citi|Citibank|Citigroup|citi|citibank|citigroup", "", txt)
    #commented code removes "hello. hello. bye?"
    #want to remove "hello \n there?"
    #txt = re.sub("[\.\?!](.*?\?)", "", txt, re.MULTILINE, re.DOTALL)
    #naive solution
    txt = re.sub(".*\?", "", txt)
    txt = re.sub("!", ".", txt)
    txt = re.sub("\r|\n", " ", txt)
    
    #Remove all letters that are not from ASCII 0-127
    #However, would remove things like trademark and registered
    txt = re.sub("[^\x00-\x7f]", "", txt)
    
    txt = re.sub(" +", " ", txt)
    txt = txt.strip()
    
    return txt

In [83]:
path = './Disruptive Innovation Text/*.txt' 
files = glob.glob(path)
# iterate over the list getting each file 
articles_list = []

#inside for loop we remove the global redundants
for fle in files:
    with codecs.open(fle, "r",encoding='utf-8', errors='ignore') as f:
        article_content = ''
        for line in f:
            article_content += line
        article_content = remove_global_redundancies(article_content)
        articles_list.append(article_content)
        
        
articles_dict = {'article_text': articles_list}
df = pd.DataFrame.from_dict(articles_dict)
 

6. eSports 
The Rise of the 
Professional Gamers 


eSports (also 
known as electronic sports 
or professional video gaming) is a 
form of 
competition using video 
games. Most commonly, eSports take the form 
of 
organized, multiplayer video 
game 
competitions 
between professional players. 
Similar to traditional sporting events like 
the 
FIFAWorld Cup, sports fans 
get 
together to watch their favorite professional teams compete 
while several millions 
also tune in 
to watch 
a live 
stream 
or highlights 
of the 
event. 
These video game 
tournaments 
can 
be played 
across 
many devices including PCs, consoles, and 
tablets, though PCs are 
predominantly used 
in tournaments 
given their computing 
power and speed. Many genres of games 
exist that can be 
played during 
these 
tournaments 
including (1) 
First Person Shooter Games such 
as Halo 2, 
(2) 
Multiplayer Online Battle 
Arena (MOBA) games 
such as Dota 2 and League of 
Legends, 
(3) 
Strategy Games 
such as Starcraft,

In [84]:
df2.iloc[0,0]

"6. eSports \nThe Rise of the \nProfessional Gamers \n\n\neSports (also \nknown as electronic sports \nor professional video gaming) is a \nform of \ncompetition using video \ngames. Most commonly, eSports take the form \nof \norganized, multiplayer video \ngame \ncompetitions \nbetween professional players. \nSimilar to traditional sporting events like \nthe \nFIFAWorld Cup, sports fans \nget \ntogether to watch their favorite professional teams compete \nwhile several millions \nalso tune in \nto watch \na live \nstream \nor highlights \nof the \nevent. \nThese video game \ntournaments \ncan \nbe played \nacross \nmany devices including PCs, consoles, and \ntablets, though PCs are \npredominantly used \nin tournaments \ngiven their computing \npower and speed. Many genres of games \nexist that can be \nplayed during \nthese \ntournaments \nincluding (1) \nFirst Person Shooter Games such \nas Halo 2, \n(2) \nMultiplayer Online Battle \nArena (MOBA) games \nsuch as Dota 2 and League of

In [71]:
output_lst = list()

#rudimentary way to check if is txt file
for txt in os.listdir("Disruptive Innovation Text"):
    if re.match(".*\.txt$", txt) != None:
        output_lst.append(
            remove_global_redundancies(read_txt("Disruptive Innovation Text/" + txt)))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1164: invalid start byte

In [47]:
# helper function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in STOPWORDS])
    return sen_new

In [37]:
# Extract word vectors
word_embeddings = {}
f = open('./glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [40]:
#######################ONLY DO ONCE FOR THE FIRST 12 ARTICLES TO GET THE FIRST FREQ TABLE!##############

#get frequency table from text.
def get_freq_table(txt):
    words = word_tokenize(txt) #list of strings where each string is a word.
    freq_table = dict()
    
    for word in words:
        word = STEMMER.stem(word)
        if word in STOPWORDS:
            continue
        if word in freq_table:
            freq_table[word] += 1
        else:
            freq_table[word] = 1
       
    return freq_table

#create giant string which contains all articles (which had global redundancies removed prior)
overall_string = ""
for s in df['article_text']:
    overall_string += s + " "

#global word frequency table
FREQ_TABLE = get_freq_table(overall_string)

#save global word frequency table to a pickle file
pickle_out = open("FREQ_TABLE.pickle","wb")
pickle.dump(FREQ_TABLE, pickle_out)
pickle_out.close()

In [41]:
print(FREQ_TABLE)

{'6.': 1, 'esport': 50, 'rise': 17, 'profession': 7, 'gamer': 14, '(': 392, 'also': 60, 'known': 9, 'electron': 10, 'sport': 25, 'video': 8, 'game': 44, ')': 392, 'form': 13, 'competit': 22, 'use': 105, '.': 932, 'common': 4, ',': 1336, 'take': 21, 'organ': 5, 'multiplay': 2, 'player': 23, 'similar': 25, 'tradit': 38, 'event': 10, 'like': 54, 'fifaworld': 1, 'cup': 4, 'fan': 8, 'get': 10, 'togeth': 5, 'watch': 5, 'favorit': 2, 'team': 15, 'compet': 5, 'sever': 12, 'million': 40, 'tune': 2, 'live': 9, 'stream': 7, 'highlight': 7, 'tournament': 7, 'play': 8, 'across': 17, 'mani': 33, 'devic': 29, 'includ': 52, 'pcs': 2, 'consol': 1, 'tablet': 1, 'though': 3, 'predomin': 2, 'given': 25, 'comput': 10, 'power': 38, 'speed': 14, 'genr': 3, 'exist': 27, 'dure': 5, '1': 40, 'first': 42, 'person': 11, 'shooter': 1, 'halo': 2, '2': 35, 'onlin': 6, 'battl': 3, 'arena': 1, 'moba': 1, 'dota': 3, 'leagu': 20, 'legend': 6, '3': 27, 'strategi': 20, 'starcraft': 2, '4': 17, 'fight': 2, '5': 19, 'race':

In [43]:
#load freq table from pkl file
def load_freq_table(pkl_dir): #wishful thinking
    pickle_off = open(pkl_dir,"rb")
    unpickled_file = pickle.load(pickle_off)
    return unpickled_file


In [45]:
load_freq_table('FREQ_TABLE.pickle')

{'6.': 1,
 'esport': 50,
 'rise': 17,
 'profession': 7,
 'gamer': 14,
 '(': 392,
 'also': 60,
 'known': 9,
 'electron': 10,
 'sport': 25,
 'video': 8,
 'game': 44,
 ')': 392,
 'form': 13,
 'competit': 22,
 'use': 105,
 '.': 932,
 'common': 4,
 ',': 1336,
 'take': 21,
 'organ': 5,
 'multiplay': 2,
 'player': 23,
 'similar': 25,
 'tradit': 38,
 'event': 10,
 'like': 54,
 'fifaworld': 1,
 'cup': 4,
 'fan': 8,
 'get': 10,
 'togeth': 5,
 'watch': 5,
 'favorit': 2,
 'team': 15,
 'compet': 5,
 'sever': 12,
 'million': 40,
 'tune': 2,
 'live': 9,
 'stream': 7,
 'highlight': 7,
 'tournament': 7,
 'play': 8,
 'across': 17,
 'mani': 33,
 'devic': 29,
 'includ': 52,
 'pcs': 2,
 'consol': 1,
 'tablet': 1,
 'though': 3,
 'predomin': 2,
 'given': 25,
 'comput': 10,
 'power': 38,
 'speed': 14,
 'genr': 3,
 'exist': 27,
 'dure': 5,
 '1': 40,
 'first': 42,
 'person': 11,
 'shooter': 1,
 'halo': 2,
 '2': 35,
 'onlin': 6,
 'battl': 3,
 'arena': 1,
 'moba': 1,
 'dota': 3,
 'leagu': 20,
 'legend': 6,
 '3': 

In [ ]:
#update freq table pkl file
#pre-con: pkl_dir is directory of pkl file, txt is a text file with global redundancies removed, whereby
#         we update the freq table counts by the words in txt
def update_freq_table(pkl_dir, txt):
    #read pkl_dir wishful thinking
    freq_table = load_freq_table(pkl_dir)
    #list of words
    words_in_txt = word_tokenize(txt)
    
    for word in words_in_txt:
        stemmed_word = STEMMER.stem(word)
        if stemmed_word in STOPWORDS:
            continue
        if word in freq_table:
            freq_table[stemmed_word] += 1
        else:
            freq_table[stemmed_word] = 1
       
    #write_freq_table wishful thinking
    #write_freq_table(pkl_dir, words_in_txt)

In [63]:
#pre-cond: txt is a text with global redundancies removed. pkl_dir is the directory of the freq_table pickle.

def summarize_alg(txt, pkl_dir):
    sentences = nltk.sent_tokenize(txt) #list of strings where each string is a sentence
    
    #-------------clean sentences (non-global redundants)---------------------
    
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z -]", "").str.replace(" +", " ")
    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    #-------------------------1st algo------------------------
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
        
    # similarity matrix initialization
    sim_mat = np.zeros([len(sentences), len(sentences)])
    
    # updating matrix coeffs
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                                  sentence_vectors[j].reshape(1,100))[0,0]
    
    # graphs
    nx_graph = nx.from_numpy_matrix(sim_mat)
    first_algo_scores = nx.pagerank_numpy(nx_graph, alpha=0.85, personalization=None, weight='weight', dangling=None)
    
    #normalize across first_algo_scores such that every score runs from [0,1].
    first_algo_scores = {k: (np.arctan(v)/np.pi + 1/2) for k, v in first_algo_scores.items()}
    
    #--------------------2nd algo-----------------------------------------------
    #load freq table
    freq_table = load_freq_table(pkl_dir) #wishful thinking
    
    second_algo_scores = dict()
    
    for i in range(len(clean_sentences)):
        sentence = clean_sentences[i]
        words_in_sentence = word_tokenize(sentence)
        second_algo_scores[i] = 0
        for word in words_in_sentence:
            if STEMMER.stem(word) in freq_table: #we only consider words which appear in freq_table.
                score_of_word = freq_table[STEMMER.stem(word)]
                if i in second_algo_scores.keys():
                    second_algo_scores[i] += score_of_word
                else:
                    second_algo_scores[i] = score_of_word
        
        #divide by length of sentence
        second_algo_scores[i] /= len(sentence)
        
    #normalize across second_algo_scores such that every score runs from [0,1].
    max_score = max(second_algo_scores.values())
    second_algo_scores = {k: v/max_score for k, v in second_algo_scores.items()}
    
    #----------------------combine scores----------------------------
    combined_scores = {i: first_algo_scores[i]*second_algo_scores[i] for i in range(len(sentences))}
    
    #----------------------extract the sentences we need-----------------------------------
    #list of tuples (index, score) sorted by score
    scores_sorted = list(sorted(combined_scores.items(), key=lambda x:-x[1]))
    
    extracted_sentences = list()
    #first article no trailing " ". so initialize to -1
    character_count = -1
    for (index, score) in scores_sorted:
        if character_count + len(sentences[index]) > 500:
            break
        else:
            extracted_sentences.append((index, sentences[index]))
            character_count += len(sentences[index])+1 #+1 to take into account the trailing " "

    #-----------return the sentences in the order they appeared as a single string--------
    final_text = functools.reduce(lambda x,y : x+" "+y, map(lambda x: x[1], sorted(extracted_sentences)))
    
    return final_text
    

In [101]:
print(summarize_alg(df.iloc[0,0],'FREQ_TABLE.pickle'))

6. eSports The Rise of the Professional Gamers eSports (also known as electronic sports or professional video gaming) is a form of competition using video games. In 2002, as the player base expanded globally, Major League Gaming was formed to promote video games as a sport. The total prize money of all eSports events held in 2017 reached $112 million, breaking the $100 million mark for the first time.


In [77]:
df.iloc[0,0]

"6. eSports The Rise of the Professional Gamers eSports (also known as electronic sports or professional video gaming) is a form of competition using video games. Most commonly, eSports take the form of organized, multiplayer video game competitions between professional players. Similar to traditional sporting events like the FIFAWorld Cup, sports fans get together to watch their favorite professional teams compete while several millions also tune in to watch a live stream or highlights of the event. These video game tournaments can be played across many devices including PCs, consoles, and tablets, though PCs are predominantly used in tournaments given their computing power and speed. Many genres of games exist that can be played during these tournaments including (1) First Person Shooter Games such as Halo 2, (2) Multiplayer Online Battle Arena (MOBA) games such as Dota 2 and League of Legends, (3) Strategy Games such as Starcraft, (4) Fighting Games, (5) Sports Games, and (5) Racing

In [100]:
a = df2.iloc[0,0]

remove_global_redundancies(a)

False


"6. eSports The Rise of the Professional Gamers eSports (also known as electronic sports or professional video gaming) is a form of competition using video games. Most commonly, eSports take the form of organized, multiplayer video game competitions between professional players. Similar to traditional sporting events like the FIFAWorld Cup, sports fans get together to watch their favorite professional teams compete while several millions also tune in to watch a live stream or highlights of the event. These video game tournaments can be played across many devices including PCs, consoles, and tablets, though PCs are predominantly used in tournaments given their computing power and speed. Many genres of games exist that can be played during these tournaments including (1) First Person Shooter Games such as Halo 2, (2) Multiplayer Online Battle Arena (MOBA) games such as Dota 2 and League of Legends, (3) Strategy Games such as Starcraft, (4) Fighting Games, (5) Sports Games, and (5) Racing

In [97]:
txt = re.sub(".*[0-9]+.*?\nCitigroup.*", "", "2018\nCitigroup")
print(txt)